### Basic Imports

In [144]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np
from tqdm import tqdm

### Env Vars

In [145]:
QUBITS_NUM = 4 
N = 16
K = 4
NUM_SHOTS = 1024
NUM_ITERATIONS = 10

w_vector = np.asarray([4,3,2,1])

### Simulator Backend

In [146]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [147]:
from scipy.optimize import minimize

### K input states (computational basis)

In [148]:
from utiles import *

In [149]:
input_states = get_first_k_eigenvectors_from_n_computational_basis(K, N)

### Ansatz State

In [150]:
from ansatz_circuit_item2 import get_full_variational_quantum_circuit

In [151]:
init_circuit_params = {
    "thetas": np.random.uniform(low=0, high=2*np.pi, size=8),
    "phis": np.random.uniform(low=0, high=2*np.pi, size=4),
    "D1": 2,
    "D2": 8
}

In [152]:
def prepare_circuit_params(thetas) -> Dict:
     return {
    "thetas": thetas[4:],
    "phis": thetas[:4],
    "D1": 2,
    "D2": 8
     }

In [153]:
def get_ansatz_state(circuit_params, input_state):
    circuit_params_with_input_state = {**circuit_params, "input_state": input_state}
    return get_full_variational_quantum_circuit(**circuit_params_with_input_state)

## Expectation Value

### convert hamiltonian to pauli strings

In [172]:
def transfrom_hamiltonian_into_pauli_strings(hamiltonian) -> List:
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_coeffs, pauli_strings

### pauli string reduction to sigma_z's

In [173]:
from qiskit.circuit.library.standard_gates import HGate, SGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [174]:
reducing_to_pauli_z_mapping = {
    'I': 'I',
    'Z': 'Z',
    'X': 'Z',
    'Y': 'Z'
} 

In [175]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string) -> str:
    reduced_pauli_string = ""
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = str(pauli_string[matrix_index])
        reduced_pauli_matrix = reducing_to_pauli_z_mapping[pauli_matrix]
        reduced_pauli_string = reduced_pauli_matrix + reduced_pauli_string
    
    return reduced_pauli_string

In [176]:
def add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string, quantum_circuit):
    quantum_registers = QuantumRegister(QUBITS_NUM, name="qubit")
    additional_circuit_layer = QuantumCircuit(quantum_registers)
    
    for quantum_register_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
        if pauli_string == "Y":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
            additional_circuit_layer.append(SGate(), [quantum_registers[quantum_register_index]])
                
    extended_quantum_circuit = quantum_circuit.compose(additional_circuit_layer)
    return extended_quantum_circuit

### probabilities distribution

In [177]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [178]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def convert_pauli_string_into_str(pauli_string) -> str:
    return str(pauli_string)

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, ansatz_circuit):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    pauli_string_str = convert_pauli_string_into_str(pauli_string)
    extended_ansatz_circuit = add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string_str, ansatz_circuit)
    probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(extended_ansatz_circuit)
    
    reduced_pauli_string = reduce_pauli_matrixes_into_sigma_z(pauli_string)
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

In [179]:
def get_expectation_value(ansatz_circuit, pauli_coeffs, pauli_strings):
    total_expection_value = 0
    
    for pauli_coeff, pauli_string in zip(pauli_coeffs, pauli_strings):
        total_expection_value += pauli_coeff*calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                    pauli_string, ansatz_circuit)
    
    return total_expection_value

## Objective Function

In [180]:
from qiskit import assemble, transpile

def cost_function(thetas, hamiltonian):
    L_w = 0
    circuit_params = prepare_circuit_params(thetas)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
    
    for j in tqdm(range(K)):
        ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[j])
        L_w += w_vector[j]*get_expectation_value(ansatz_state, pauli_coeffs, pauli_strings)
        
    return L_w

### Optimization

In [181]:
def get_optimal_thetas_of_ansatz_circuit_for_hamiltonian(hamiltonian):
    initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
    optimizer_result = minimize(cost_function,x0=initial_thetas,args=(hamiltonian),method="BFGS",options={"maxiter":NUM_ITERATIONS})
    optimal_thetas = prepare_circuit_params(optimizer_result.x)
    
    return optimal_thetas

In [182]:
def get_approximated_k_eigenvalues_of_hamiltonian(hamiltonian):
    approximated_k_eigenvalues = []
    optimal_thetas = get_optimal_thetas_of_ansatz_circuit_for_hamiltonian(hamiltonian)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_strings(hamiltonian)
    for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
        optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
        approximated_eigenvalue = get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings)
        approximated_k_eigenvalues.append(approximated_eigenvalue)

    return approximated_k_eigenvalues

## Comparsion

In [183]:
from numpy import linalg as LA
from statistics import mean

def get_mean_approximation_error(exact_k_eigenvalues, approximated_k_eigenvalues):
    approximated_errors = []
    for exact_eigen_value, approximated_eigen_value in zip(exact_k_eigenvalues, approximated_k_eigenvalues):
        approximated_errors.append(abs(abs(exact_eigen_value)-abs(approximated_eigen_value))/abs(exact_eigen_value))
        
    return mean(approximated_k_eigenvalues)

In [184]:
def get_exact_k_eigenvalues_of_hamiltonian(hamiltonian, k):
    eigenvalues = LA.eig(hamiltonian.to_matrix())[0]
    
    return sorted(eigenvalues,reverse=True)[:k-1]

In [185]:
def compare_exact_and_approximated_eigenvectors(hamiltonian, approximated_k_eigenvalues):
    exact_k_eigenvalues = get_exact_k_eigenvalues_of_hamiltonian(hamiltonian, K)
    print("Exact K Eigenvalues:")
    print(exact_k_eigenvalues)
    print("Approximated K Eigenvalues:")
    print(approximated_k_eigenvalues)

    print("Approximation error")
    print(get_mean_approximation_error(exact_k_eigenvalues, approximated_k_eigenvalues))

## Visualization

In [186]:
def insert_approximated_energy_to_list_of_all_approximated_energies(approximated_energies_list, energy):
    approximated_energies_list.append(energy)

## LiH Molecule 4 qubits

In [187]:
from hamitonians import *

In [ ]:
%%time
LiH_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(LiH_molecule_4_qubits)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

In [ ]:
compare_exact_and_approximated_eigenvectors(LiH_molecule_4_qubits, LiH_approximated_k_eigenvalues)

## H2 Molecule 4 qubits

In [ ]:
%%time
H2_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(H2_molecule_Hamiltonian_4_qubits)

In [ ]:
compare_exact_and_approximated_eigenvectors(H2_molecule_Hamiltonian_4_qubits, H2_approximated_k_eigenvalues)

##  Transverse Ising Model 4 qubits

In [ ]:
%%time
TI_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(transverse_ising_4_qubits)

In [ ]:
compare_exact_and_approximated_eigenvectors(transverse_ising_4_qubits, TI_approximated_k_eigenvalues)

In [113]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
optimizer_result = minimize(cost_function,x0=initial_thetas,args=(LiH_molecule_4_qubits),method="BFGS",options={"maxiter":NUM_ITERATIONS})
print(optimizer_result)
optimal_thetas = prepare_circuit_params(optimizer_result.x)

computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
optimal_ansatz_state = get_ansatz_state(optimal_thetas, computational_eigenvectors[K-1])
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.28s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.60s/it]

      fun: -73.95773316051425
 hess_inv: array([[ 1.00307161e+00, -5.09201222e-02, -5.47356480e-02,
        -1.62681403e-01, -4.23696808e-02,  1.91395552e-01,
        -2.34563415e-01, -1.47399387e-02, -1.06994899e-01,
        -2.30856804e-01,  2.44203224e-01, -2.78136077e-03],
       [-5.09201222e-02,  8.96386378e-01, -9.54867683e-02,
        -1.71902098e-01, -6.07276602e-02,  9.78304586e-02,
        -2.22414166e-01, -6.01544432e-02, -1.33741982e-01,
        -2.23651086e-01,  1.34317906e-01, -3.42505201e-02],
       [-5.47356480e-02, -9.54867683e-02,  9.14173327e-01,
        -1.36682596e-01, -5.24829952e-02,  5.03213518e-02,
        -1.70152705e-01, -5.95562678e-02, -1.11781260e-01,
        -1.72491842e-01,  7.42529145e-02, -3.54695988e-02],
       [-1.62681403e-01, -1.71902098e-01, -1.36682596e-01,
         9.32466442e-01, -6.58881008e-02, -2.36598771e-01,
        -2.03558696e-02, -1.41056904e-01, -1.07021666e-01,
        -3.44168301e-02, -2.73137076e-01, -9.59325722e-02],
       [-4.

In [114]:
print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))
print(optimal_thetas)

-7.57072049639962
{'thetas': array([0.74760824, 0.17289584, 1.48511553, 2.87050967, 1.1083423 ,
       3.83152585, 4.61583078, 3.2915738 ]), 'phis': array([4.29275154, 5.47569434, 2.44077279, 1.02772525]), 'D1': 2, 'D2': 8}


In [115]:
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
    print(eigenvector)
    optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
    print("eigenvalue for k =", eigenvalue_index)
    print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 0
-7.162165959477899
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 1
-7.517719632622515
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 2
-7.589285715860265
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 3
-7.563737559574652


In [120]:
from numpy import linalg as LA
eigen_values = LA.eig(LiH_molecule_4_qubits.to_matrix())[0]
print(sorted(eigen_values, reverse=True))

[(-6.769813218087976+0j), (-7.130406955301308+0j), (-7.130406955301309+0j), (-7.151525481896562+0j), (-7.364817440287081+0j), (-7.511999706834451+0j), (-7.511999706834453+0j), (-7.569984737620559+0j), (-7.569984737620559+0j), (-7.700475837803976+0j), (-7.714056691660695+0j), (-7.714056691660695+0j), (-7.7140566916607005+0j), (-7.783396208286518+0j), (-7.7833962082865185+0j), (-7.862773163027979+0j)]
